# Using Tweepy to Create Networks

This notebook contains examples for using web-based APIs (Application Programmer Interfaces) to download data from social media platforms.

This notebook focuses specifically on _Twitter_.

We will use this API to create a network from the Twitter data, using the friend/follower graph.

In [ ]:
# First we need to make sure we have tweepy installed...
!pip install tweepy

In [ ]:
%matplotlib inline

import time
import json

<hr>

Import the `NetworkX` package

In [ ]:
# First we need to make sure we have networkx installed...
!pip install networkx

import networkx as nx

<hr>

## Twitter API

Twitter's API is useful and flexible but takes several steps to configure. 
To get access to the API, you first need to have a Twitter account and have a mobile phone number (or any number that can receive text messages) attached to that account.
Then, we'll use Twitter's developer portal to create an "app" that will then give us the keys and tokens (essentially IDs and passwords) we will need to connect to the API.

So, in summary, the general steps are:

0. Have a Twitter account,
1. Configure your Twitter account with your mobile number,
2. Create an app on Twitter's developer site, and
3. Generate consumer and access keys and secrets or _bearer tokens_.

We will then plug these strings into the code below.

In [ ]:
# For our first piece of code, we need to import the package 
# that connects to Twitter. Tweepy is a popular and fully featured
# implementation.

import tweepy

### Creating Twitter Credentials

For more in-depth instructions for creating a Twitter account and/or setting up a Twitter account to use the following code, I have provided content in the lecture.

You can also visit [this Medium post](https://towardsdatascience.com/ultimate-beginners-guide-to-collecting-text-for-natural-language-processing-nlp-with-python-256d113e6184) for a good overview of several data collection approaches or [this Twitter-specific Medium post](https://towardsdatascience.com/how-to-access-twitters-api-using-tweepy-5a13a206683b) for a slightly outdated version.


In [ ]:
# Use the strings from your Twitter app webpage to populate these  
# variables. Be sure and put the strings BETWEEN the quotation marks
# to make it a valid Python string.

api_key = "xxx"
api_secret = "xxx"
bearer_token = "xxx"


### Connecting to Twitter

Once we have the authentication details set, we can connect to Twitter using the Tweepy OAuth handler, as below.

In [ ]:
# Now we use the configured authentication information to connect
# to Twitter's API
auth = tweepy.AppAuthHandler(api_key, api_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

print("Connected to Twitter!")

In [ ]:
g = nx.Graph()

target = "codybuntain"
total_friends = 20

# Get the first few friends of mine and first few of each of them
#  and add their links to the graph
for friend in tweepy.Cursor(api.friends, id=target).items(total_friends):
    g.add_node(friend.screen_name)
    g.add_edge(target, friend.screen_name)
    print("Processing:", friend.screen_name)
    
    for friend_of_friend in tweepy.Cursor(api.friends, id=friend.screen_name).items(total_friends):
        g.add_node(friend_of_friend.screen_name)
        g.add_edge(friend.screen_name, friend_of_friend.screen_name)
        print("\t->", friend_of_friend.screen_name)

### Export Graph

We export the graph using GraphML in `NetworkX`, so we can load it in other software later.

Note, we could use other formats here as well. GraphML is just convenient.

In [ ]:
nx.write_graphml(g, "output.twitter.graphml", prettyprint=False)

### Draw the Graph

Now that we've made the graph, let's draw it using the layout algorithms in `NetworkX`.

_NOTE_: `NetworkX` is not meant for graph layouts. We only do this for illustrative purposes.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Use the Spring layout algorithm
pos = nx.spring_layout(g, scale=200, iterations=100, k=0.2)

# And draw the graph with node labels
nx.draw(g, 
        pos, 
        node_color='#A0CBE2', 
        width=1, 
        with_labels=True,
        node_size=50)